PDB Extraction Tool


In [2]:
pip install biopython scipy


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from Bio.PDB import PDBParser, is_aa
from scipy.spatial import cKDTree
from collections import defaultdict
import os

# === INPUT ===
pdb_path = r"C:\Users\shirk\Documents\Github\AclR\AclR_PISA-Extraction\Inputs\PISA_interfaces\AclR-w-C8-ACL_Phil2.pdb"
ligand_cutoff = 4.0
interface_cutoff = 4.0

# === PARSE STRUCTURE ===
parser = PDBParser(QUIET=True)
structure = parser.get_structure("AclR", pdb_path)
model = structure[0]

# === STEP 1: Identify Chains and Classify Them ===
chain_atom_counts = {}
chain_residues = {}
for chain in model:
    atom_count = sum(1 for res in chain for atom in res)
    resnames = set(res.resname for res in chain)
    chain_atom_counts[chain.id] = atom_count
    chain_residues[chain.id] = [res for res in chain]

# Heuristic: chains with many atoms and standard residues are protein
protein_chains = []
ligand_chains = []

for cid, residues in chain_residues.items():
    if all(not is_aa(res) for res in residues):  # non-amino acid
        ligand_chains.append(cid)
    elif any(is_aa(res) for res in residues):  # has amino acids
        protein_chains.append(cid)

print(f"Protein Chains: {protein_chains}")
print(f"Ligand Chains: {ligand_chains}")

# === STEP 2: Ligand Contact Residues (≤4 Å) ===
ligand_contacts = set()

for lig_cid in ligand_chains:
    ligand_atoms = [atom for res in model[lig_cid] for atom in res]
    ligand_coords = [atom.coord for atom in ligand_atoms]
    ligand_tree = cKDTree(ligand_coords)

    for prot_cid in protein_chains:
        for res in model[prot_cid]:
            if not is_aa(res):
                continue
            for atom in res:
                dist, _ = ligand_tree.query(atom.coord, distance_upper_bound=ligand_cutoff)
                if dist != float('inf'):
                    rid = f"{prot_cid}:{res.resname} {res.id[1]}"
                    ligand_contacts.add(rid)
                    break  # one contact is enough

print("\nResidues within 4Å of ligand:")
for res in sorted(ligand_contacts):
    print(res)

# === STEP 3: Dimer Interface Residues (≤5 Å) ===
interface_residues = set()

for i in range(len(protein_chains)):
    for j in range(i + 1, len(protein_chains)):
        cid1, cid2 = protein_chains[i], protein_chains[j]

        atoms1 = [(res, atom) for res in model[cid1] if is_aa(res) for atom in res]
        atoms2 = [(res, atom) for res in model[cid2] if is_aa(res) for atom in res]

        coords2 = [atom.coord for _, atom in atoms2]
        tree2 = cKDTree(coords2)

        for res1, atom1 in atoms1:
            dist, idx = tree2.query(atom1.coord, distance_upper_bound=interface_cutoff)
            if dist != float("inf"):
                res2, _ = atoms2[idx]
                r1 = f"{cid1}:{res1.resname} {res1.id[1]}"
                r2 = f"{cid2}:{res2.resname} {res2.id[1]}"
                interface_residues.add(r1)
                interface_residues.add(r2)

print("\nDimer Interface Residues (≤5Å):")
for res in sorted(interface_residues):
    print(res)


Protein Chains: ['A', 'B']
Ligand Chains: ['G', 'C']

Residues within 4Å of ligand:
A:ARG 11
A:ASN 12
A:GLU 171
A:ILE 155
A:ILE 43
A:LYS 173
A:LYS 41
A:SER 154
A:SER 42
A:SER 57
A:SER 9
A:THR 188
A:TYR 40
A:TYR 44
B:GLU 171
B:HIS 113
B:ILE 105
B:LYS 104
B:LYS 168
B:LYS 173
B:PHE 101
B:PHE 175
B:PRO 106

Dimer Interface Residues (≤5Å):
A:GLU 171
A:ILE 105
A:LYS 104
A:LYS 185
A:LYS 250
A:PHE 101
A:PHE 175
A:PHE 183
A:PRO 106
A:THR 188
A:TRP 184
A:VAL 186
B:ASN 172
B:GLU 171
B:GLU 191
B:LYS 104
B:LYS 173
B:LYS 185
B:PHE 175
B:PHE 183
B:PRO 106
B:THR 188
B:TRP 184
B:VAL 186
